<a href="https://colab.research.google.com/github/Sharko123/AppliedResearchOAfa2025/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline
from pathlib import Path
import requests
import tempfile
import os
import gradio as gr
import numpy as np
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Model
import torch
import librosa

pipe = pipeline("audio-classification", model="dima806/music_genres_classification")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the model checkpoint at dima806/music_genres_classification were not used when initializing Wav2Vec2ForSequenceClassification: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrai

In [12]:
!pip install librosa

In [24]:
def classify_files(uploaded_files):
    if uploaded_files:
        results =[]
        path = uploaded_files.name
        y, sr = librosa.load(path, sr=16000)  # waveform as np.ndarray
        res = pipe(y, sampling_rate=sr)
        top_label = max(res, key=lambda x: x['score'])['label']
        results.append(f"Estimated Genre: {top_label}\n")
        bpm = estimate_bpm(path)
        embedding = get_embedding(path)
        results.append(f"Estimated BPM: {bpm}\n")
        results.append(f"Embedding: {embedding[:10]}\n") #only printing first 10 values of embedding array
        key = get_key(path)
        results.append(f"Estimated Key: {key}\n")

        #similarity checker
        min = 1000000000;
        for row in ds:
          if row['bpm'] is None or row['key'] is None:
            continue  # skip rows with missing data
          score = similarity_score(bpm, row['bpm'], key, row['key'])
          if score < min:
            best_row = row
            min = score
        results.append(f"Best Match: {best_row['main_caption']}\n")
        results.append(f"Best Match File Location: {best_row['location']}\n")

        return "\n".join(results)


In [3]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
def get_key(audio_path):
    y, sr = librosa.load(audio_path, sr=16000)
    chromagram = librosa.feature.chroma_stft(y=y, sr=sr)
    mean_chroma = np.mean(chromagram, axis=1)
    chroma_to_key = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] #map the chroma value to a key from this list
    estimated_key_index = np.argmax(mean_chroma)
    estimated_key = chroma_to_key[estimated_key_index]
    return estimated_key
def get_embedding(audio_path):
    y, sr = librosa.load(audio_path, sr=16000) #auto sampling rate of 16000
    input_values = feature_extractor(y, sampling_rate=sr, return_tensors="pt").input_values
    with torch.no_grad():
        embeddings = model(input_values).last_hidden_state.mean(dim=1) # generate embeddings using HG model
    return embeddings[0].numpy()

def estimate_bpm(audio_path): #simple BPM calculator
    y, sr = librosa.load(audio_path)
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
    print(tempo)
    return round(tempo[0], 2)

In [5]:
def similarity_score(bpm, bpm1, key, key1):
  key_score = 1 if str(key) == str(key1) else 0
  return (bpm - bpm1)*0.3 + key_score*0.7 #added weights becuase similar keys is more likely and important than similar BPMs. BPMs also range more.

In [6]:
from datasets import load_dataset

ds = load_dataset("amaai-lab/MusicBench",split="train", streaming=True) #dataset of other song BPM and keys to compare to and see which is similar

In [25]:
iface = gr.Interface( #gr interface for UI of uploading files
    fn=classify_files,
    inputs=[
        gr.File(file_types=[".mp3", ".wav"]),
    ],
    outputs=gr.Textbox(label = "Analysis Result"),
    title="Music Genre Classifier"
)

iface.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://d6c6f8f05b1d79d805.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


[95.703125]
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://d6c6f8f05b1d79d805.gradio.live
